In [ ]:
'''
@Author:Haihui Pan
@Date: 2021-11-5
@Ref: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
'''

# Optimizing Model parameters

* 训练模型是一个迭代的过程，在每一次迭代中都会计算loss对于参数的梯度，之后使用梯度来更新这些参数。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda

## 加载数据

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

D:\software\Anaconda\envs\env1\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 构建模型

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 模型训练&评估

In [7]:
#超参数
learning_rate=1e-3
batch_size=64
epochs=5

#定义损失函数
loss_fn=nn.CrossEntropyLoss()

#定义优化器
optimizer =torch.optim.SGD(model.parameters(),lr=learning_rate)

#定义单个epoch的训练过程
def train_loop(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    
    for batch,(X,y) in enumerate(dataloader):
        #梯度清零
        optimizer.zero_grad()
        
        #前馈计算
        pred=model(X)
        #计算loss
        loss=loss_fn(pred,y)
        
        #反向传播
        loss.backward()
        #更新参数
        optimizer.step()
        
        if batch%100==0:
            loss,current=loss.item(),batch*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#定义测试集上的评估
def test_loop(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss,correct=0,0
    
    with torch.no_grad():
        for X,y in dataloader:
            #前馈计算
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
#模型训练
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.165796  [    0/60000]
loss: 2.153887  [ 6400/60000]
loss: 2.091163  [12800/60000]
loss: 2.103169  [19200/60000]
loss: 2.049761  [25600/60000]
loss: 1.991176  [32000/60000]
loss: 2.027802  [38400/60000]
loss: 1.938788  [44800/60000]
loss: 1.950355  [51200/60000]
loss: 1.881411  [57600/60000]
Test Error: 
 Accuracy: 54.1%, Avg loss: 1.870502 

Epoch 2
-------------------------------
loss: 1.914008  [    0/60000]
loss: 1.883459  [ 6400/60000]
loss: 1.755084  [12800/60000]
loss: 1.794374  [19200/60000]
loss: 1.682858  [25600/60000]
loss: 1.638047  [32000/60000]
loss: 1.665849  [38400/60000]
loss: 1.557462  [44800/60000]
loss: 1.583738  [51200/60000]
loss: 1.490677  [57600/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.500915 

Epoch 3
-------------------------------
loss: 1.575554  [    0/60000]
loss: 1.542279  [ 6400/60000]
loss: 1.384044  [12800/60000]
loss: 1.458289  [19200/60000]
loss: 1.337915  [25600/60000]
loss: 1.333395  [32000/600